# 1901042697 Muhammed Bedir ULUCAY

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
from sklearn import metrics

# Audit Dataset

In [ ]:
audit_risk = pd.read_csv("audit_data/audit_risk.csv")
trial = pd.read_csv("audit_data/trial.csv")

trial.columns = ['Sector_score','LOCATION_ID', 'PARA_A', 'Score_A', 'PARA_B', 'Score_B',  'TOTAL', 'numbers', 'Marks', 'Money_Value', 'MONEY_Marks', 'District', 'Loss', 'LOSS_SCORE', 'History', 'History_score', 'Score', 'Risk_trial' ]
trial['Score_A'] = trial['Score_A']/10
trial['Score_B'] = trial['Score_B']/10
merged_df = pd.merge(audit_risk, trial, how='outer', on = ['History', 'LOCATION_ID', 'Money_Value', 'PARA_A', 'PARA_B', 'Score', 'Score_A', 'Score_B', 'Sector_score', 'TOTAL', 'numbers'])

df = merged_df.drop(['Risk_trial', 'Detection_Risk', 'Risk_F'], axis = 1)
df['Money_Value'] = df['Money_Value'].fillna(df['Money_Value'].median())
df = df[(df.LOCATION_ID != 'LOHARU')]
df = df[(df.LOCATION_ID != 'NUH')]
df = df[(df.LOCATION_ID != 'SAFIDON')]
df = df.astype(float)
df = df.drop_duplicates(keep = 'first')
df = df[['Risk_A', 'Risk_B', 'Risk_C', 'Risk_D','RiSk_E', 'Prob', 'Score', 'CONTROL_RISK', 'Audit_Risk', 'MONEY_Marks', 'Loss', 'Risk']]
df = df.rename(columns={"Risk": "label"})
audit_class_df = df.drop("Audit_Risk", axis = 1)
audit_class_df.info()
audit_class_df.to_csv("audit_data/audit_clean_data.csv", index=False)
print("Updated number of rows in the dataset: ",len(df))

# Bike Sharing Dataset

In [ ]:
day_df = pd.read_csv('Bike-Sharing-Dataset/day.csv')
hour_df = pd.read_csv('Bike-Sharing-Dataset/hour.csv')

day_df.rename(columns={'instant':'id','dteday':'Date','yr':'Year','mnth':'Month',
                       'weathersit':'WeatherCondition','atemp':'FeelinTemp',
                       'hum':'Humidity','cnt':'label'},inplace=True)
# hour_df.rename(columns={'instant':'id','dteday':'Date','yr':'Year','mnth':'Month','hr':'Hours', 'weathersit':'WeatherCondition','atemp':'FeelinTemp','hum':'Humidity','cnt':'TotalRentHourly'},inplace=True)
hour_df.rename(columns={'instant':'rec_id','dteday':'datetime','holiday':'is_holiday',
                     'workingday':'is_workingday', 'weathersit':'weather_condition',
                     'hum':'humidity','mnth':'month', 'cnt':'label','hr':'hour',
                     'yr':'year'},inplace=True)

day_df['Date']=pd.to_datetime(day_df.Date)
day_df['season']=day_df.season.astype('category')
day_df['Year']=day_df.Year.astype('category')
day_df['Month']=day_df.Month.astype('category')
day_df['holiday']=day_df.holiday.astype('category')
day_df['weekday']=day_df.weekday.astype('category')
day_df['workingday']=day_df.workingday.astype('category')
day_df['WeatherCondition']=day_df.WeatherCondition.astype('category')
day_df = day_df.drop(['Date'], axis=1)

hour_df['datetime']=pd.to_datetime(hour_df.datetime)
hour_df['season']=hour_df.season.astype('category')
hour_df['year']=hour_df.year.astype('category')
hour_df['month']=hour_df.month.astype('category')
hour_df['hour']=hour_df.hour.astype('category')
hour_df['is_holiday']=hour_df.is_holiday.astype('category')
hour_df['weekday']=hour_df.weekday.astype('category')
hour_df['is_workingday']=hour_df.is_workingday.astype('category')
hour_df['weather_condition']=hour_df.weather_condition.astype('category')
hour_df = hour_df.drop(['datetime'], axis=1)

print("DaysRents_df: {}\nHourlyRents_df: {}".format(day_df.shape, hour_df.shape))
print(hour_df.columns)
print(day_df.columns)

hour_df.info()
day_df.info()


# Decision Tree Classification

In [ ]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df


def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    # condition check
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    if not isinstance(answer, dict):
        return answer
    
    # iter branches
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)
    

def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy


In [ ]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False


def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification


def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits


def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above


def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy


def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy


def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value


def decision_tree_algorithm(df, counter=0, min_samples=10, max_depth=15):
    
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    else:    
        counter += 1

        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
def build_df(X, y, attribute_types, options):
    df = pd.DataFrame(X)
    df['label'] = y
    df = df.dropna()
    tree = decision_tree_algorithm(df, max_depth=options['max_depth'], min_samples=options['min_samples'])
    return tree


def predict_df(dt, X, options):
    accuracy = calculate_accuracy(X, dt)
    return accuracy

df = audit_class_df
random.seed(0)
train_df, test_df = train_test_split(df, test_size=50)


options = {
    'max_depth': 25,
    'min_samples': 4,
}
attribute_types = {}
tree = build_df(train_df.drop('label', axis=1), train_df['label'], None, options)
accuracy = predict_df(tree, test_df, options)

print(f"Accuracy: {accuracy}")
pprint(tree)


# Decision Tree Regression

In [ ]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False


def create_leaf(data):
    
    label_column = data[:, -1]
    leaf = np.mean(label_column)
    
    return leaf


def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits


def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above


def calculate_mse(data):
    actual_values = data[:, -1]
    if len(actual_values) == 0:   # empty data
        mse = 0
        
    else:
        prediction = np.mean(actual_values)
        mse = np.mean((actual_values - prediction) **2)
    
    return mse


def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy


def calculate_overall_metric(data_below, data_above, metric_function):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_metric =  (p_data_below * metric_function(data_below) 
                     + p_data_above * metric_function(data_above))
    
    return overall_metric


def determine_best_split(data, potential_splits):
    
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            
            current_overall_metric = calculate_overall_metric(data_below, data_above, metric_function=calculate_mse)

            if first_iteration or current_overall_metric <= best_overall_metric:
                first_iteration = False
                
                best_overall_metric = current_overall_metric
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value


def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types


def reg_decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        leaf = create_leaf(data)
        return leaf
    
    else:    
        counter += 1

        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        if len(data_below) == 0 or len(data_above) == 0:
            leaf = create_leaf(data)
            return leaf
        
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        sub_tree = {question: []}
        
        yes_answer = reg_decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = reg_decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree


def random_forest_algorithm(train_df, n_trees, n_bootstrap, n_features, dt_max_depth):
    forest = []
    for i in range(n_trees):
        df_bootstrapped = train_df.sample(n=n_bootstrap, replace=True)
        tree = reg_decision_tree_algorithm(df_bootstrapped, max_depth=dt_max_depth)
        forest.append(tree)
    return forest


In [ ]:
def build_rdf(X, y, attribute_types, N = 1, options = {"max_depth": 13, "min_samples": 2, "max_features": None}):
    tree = reg_decision_tree_algorithm(X, max_depth=options["max_depth"], min_samples=options["min_samples"])
    return tree


def predict_df(dt, X, options):
    accuracy = calculate_accuracy(X, dt)
    return accuracy

train_df, test_df = train_test_split(hour_df, test_size=15)

options = {
    "max_depth": 90,
    "min_samples": 2,
    "max_features": None,
}

tree = build_rdf(train_df, train_df['label'], None, 1, options=options)

accuracy = predict_df(tree, test_df, options)
print(f"Accuracy: {accuracy}")

pprint(tree)
